# Solving time-dependent Ising Model using Pulser

In [ ]:
import cmath
import numpy as np
import networkx as nx
from scipy.constants import hbar
from collections import Counter
from typing import Union, Literal
from networkx.drawing.nx_agraph import graphviz_layout
from qat.core import Variable, Schedule, Observable, Term, Result
from qat.core.qpu import QPUHandler
from qat.core.variables import ArithExpression, angle, cos, sin, real, imag, sqrt



## Defining an Ising Hamiltonian on MyQLM

On Pulser we can solve problems of shape 
$$ H = \hbar\sum_i \frac{\Omega(t)}{2} \sigma_i^x - \delta(t)n_i + \frac{1}{2}\sum_{i\neq j}U_{ij}n_i n_j$$
with $\sigma_i^x$ the Pauli $X$ operator applied on qubit $i$ and $n_i = \frac{1-\sigma_i^z}{2}$ with $\sigma_i^z$ the Pauli $Z$ operator applied on qubit $i$

Let's start by defining the parameters and the Hamiltonian

In [ ]:
nqubits = 4
tmax = 23.0
t_variable = Variable("t")
u_variable = Variable("u")
# Omega_t = 1 - t_variable
Omega_t = 1
delta_t = u_variable
U_matrix = [[0, 1, 0, 0],
            [1, 0, 1, 0],
            [0, 1, 0, 1],
            [0, 0, 1, 0]]

In [ ]:
hamiltonian = 0
for i in range(nqubits):
    hamiltonian += Omega_t / 2 * Observable(nqubits, pauli_terms=[Term(1, 'X', [i])])
    hamiltonian -= delta_t / 4 * Observable(nqubits, pauli_terms=[Term(1, 'I', [i]),
                                                                      Term(-1, 'Z', [i])])
    for j in range(i):
            hamiltonian += U_matrix[i][j] / 4 * Observable(nqubits, pauli_terms=[Term(1, 'II', [i, j]), 
                                                                                 Term(-1, 'IZ', [i, j]),
                                                                                 Term(-1, 'ZI', [i, j]),
                                                                                 Term(1, 'ZZ', [i, j])]) 
                                                 

The Hamiltonian is implemented as a `Schedule` object. You also have to define the duration of the evolution

In [ ]:
schedule = Schedule(drive=hamiltonian,
                    tmax=tmax)

print(schedule)

From a schedule object we can build jobs to be submit to the QPU. An initial state can be defined in the job, as well as an Hamiltonian to be measured at the output.

In [ ]:
# To simply sample the final state in the computational basis
job0 = schedule.to_job()

# To evaluate some observable at the end of the computation
H_target = Observable(nqubits, pauli_terms=[Term(1, "XX", [0, 1])])
jobobs = schedule.to_job(observable=H_target)

# Starting from |++++> state
job1 = schedule.to_job(psi_0='++++')

# Starting from |+1+1> state
job2 = schedule.to_job(psi_0='+1+1')

# Starting from a random initial state (simulator only)
vec = np.random.random(2**nqubits)
vec /= np.linalg.norm(vec)
job3 = schedule.to_job(psi_0=vec)

## Defining Pasqal's QPU

In [ ]:
from pulser import Pulse, Sequence, Register
from pulser_simulation import Simulation
from pulser.waveforms import CustomWaveform, Waveform
from pulser.devices import Device, VirtualDevice, interaction_coefficients
from pulser.devices import MockDevice
from pulserAQPU import FresnelAQPU

In [ ]:
qpu = FresnelAQPU()

In [ ]:
qpu.submit_job(job0(u=1))